In [7]:
import boto3
import pandas as pd


def load_data_s3(bucket_name='10academy-group2-bucket', file_name='Clean_Amharic.txt'):
    """ Load transcription data from s3 bucket"""
    s3 = boto3.resource(
        service_name='s3',
        region_name='us-east-1'
    )
    # Load file directly into python
    obj = s3.Bucket(bucket_name).Object(file_name).get()
    df = pd.read_csv(obj['Body'])
    return df

In [8]:
load_data_s3()

,ብርሀን ፈይሳየኢትዮጵያ ቦክስ ፌዴሬሽን በየአመቱ የሚያዘጋጀው የክለቦች ቻምፒዮና በአዲስ አበባ ከተማ በመካሄድ ላይ ይገኛል (sentence 1)
0,የተሻለ ብቃት ያሳዩ ቦክሰኞች ለቶኪዮ ኦሊምፒክ ማጣሪያ ተሳታፊ እንደሚሆ...
1,በቦክስ ስፖርት ከሚካሄዱት አመታዊ ቻምፒዮናዎች መካከል አንዱ በክለቦች መ...
2,የኢትዮጵያ ቦክስ ፌዴሬሽን ከአዲስ አበባ ቦክስ ፌዴሬሽን ጋር በመተባበር...
3,በውድድሩም ላይ አምስት ክለቦች ከአዲስ አበባ ፌዴራልፖሊስ ፌዴራል ማረሚ...
4,በአጠቃላይ ክለቦቻቸውን ወክለውም 12 ሴትና 76 ወንድ ቦክሰኞች በጥቅሉ...
...,...
662353,የ2011 በጀት አመት የውጭ ዲሎማሲያዊ ተግባራት ስኬታማ እንደነበሩ ተገለ...
662354,አቶ አገኘሁ ተሻገር የሰላም ግንባታና የህዝብ ደህንነት ሀላፊ ሆነው ተሾሙ...
662355,የአማራ ክልል ምክር ቤት የ230 ዳኞችን ሹመት አፀደቀ (sentence 6...
662356,ሀምሌ 22 ለሚካሄደው የችግኝ ተከላ መርሀግብር 54 ሚሊየን ብር ተመድቧል...


In [10]:
df = load_data_s3()

In [12]:
from pyspark.sql import SparkSession
#Create PySpark SparkSession
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("Spark_reading_s3") \
    .getOrCreate()
#Create PySpark DataFrame from Pandas
sparkDF=spark.createDataFrame(df) 
sparkDF.printSchema()
sparkDF.show()

root
 |-- ብርሀን ፈይሳየኢትዮጵያ ቦክስ ፌዴሬሽን በየአመቱ የሚያዘጋጀው የክለቦች ቻምፒዮና በአዲስ አበባ ከተማ በመካሄድ ላይ ይገኛል (sentence 1): string (nullable = true)



21/09/13 08:43:47 WARN TaskSetManager: Stage 0 contains a task of very large size (184644 KiB). The maximum recommended task size is 1000 KiB.


+------------------------------------------------------------------------------------------+
|ብርሀን ፈይሳየኢትዮጵያ ቦክስ ፌዴሬሽን በየአመቱ የሚያዘጋጀው የክለቦች ቻምፒዮና በአዲስ አበባ ከተማ በመካሄድ ላይ ይገኛል (sentence 1)|
+------------------------------------------------------------------------------------------+
|                                                                       የተሻለ ብቃት ያሳዩ ቦክሰ...|
|                                                                      በቦክስ ስፖርት ከሚካሄዱት ...|
|                                                                       የኢትዮጵያ ቦክስ ፌዴሬሽን...|
|                                                                       በውድድሩም ላይ አምስት ክ...|
|                                                                       በአጠቃላይ ክለቦቻቸውን ወ...|
|                                                                       ውድድሩ የሚካሄደው በራስ ...|
|                                                                       የቦክስ ስፖርት ከንክኪ ስ...|
|                                                                     